In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import Input
print('DONE')


DONE


In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.4/904.4 kB 16.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import os
import torch
from tqdm import tqdm
from torchvision import models, transforms
from ultralytics import YOLO
import numpy as np
from PIL import Image
import random


# Load YOLO model
yolo_model_path = "/kaggle/input/yolov5npt/yolov5su.pt"
yolo_model = YOLO(yolo_model_path)

# Load a pre-trained TSN model (e.g., ResNet-based backbone for TSN)
class TSN(torch.nn.Module):
    def __init__(self, base_model):
        super(TSN, self).__init__()
        self.base_model = base_model

    def forward(self, frames):
        batch_features = [self.base_model(frame.unsqueeze(0)) for frame in frames]
        return torch.mean(torch.stack(batch_features), dim=0)

base_model = models.resnet50(pretrained=True)
ts_model = TSN(base_model).eval()

# Image preprocessing pipeline for TSN input
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Output paths for extracted frames
output_paths = {
    "train_violence": "/kaggle/working/extracted_frames/train/violence",
    "train_nonviolence": "/kaggle/working/extracted_frames/train/nonviolence",
    "val_violence": "/kaggle/working/extracted_frames/val/violence",
    "val_nonviolence": "/kaggle/working/extracted_frames/val/nonviolence"
}

# Create output directories if they don't exist
for path in output_paths.values():
    os.makedirs(path, exist_ok=True)

# Function to extract YOLO-detected bounding box regions and TSN classification
def process_video(video_path, output_folder, target_fps=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return

    original_fps = cap.get(cv2.CAP_PROP_FPS)
    if original_fps == 0:
        print(f"Error reading FPS for video: {video_path}")
        return

    frame_interval = int(original_fps / target_fps)
    frame_count, saved_frame_count = 0, 0
    frames_for_tsn = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            # YOLO detection
            results = yolo_model(frame)
            for bbox in results[0].boxes.data.tolist():
                x1, y1, x2, y2, score, class_id = bbox
                if score > 0.5:  # Confidence threshold
                    cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
                    if cropped_frame.size > 0:
                        # Convert to PIL Image for preprocessing
                        pil_image = Image.fromarray(cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB))
                        # Preprocess frame for TSN
                        tsn_frame = preprocess(pil_image)
                        frames_for_tsn.append(tsn_frame)

            # Fuse TSN output
            if len(frames_for_tsn) >= 8:  # Process in batches of 8 frames
                with torch.no_grad():
                    tsn_features = ts_model(torch.stack(frames_for_tsn[:8]))
                predicted_class = torch.argmax(tsn_features).item()
                label = "violence" if predicted_class == 1 else "nonviolence"
                output_path = os.path.join(output_folder, label, f"frame_{saved_frame_count}.jpg")
                cv2.imwrite(output_path, frame)
                saved_frame_count += 1
                frames_for_tsn = []  # Clear frames for the next batch

        frame_count += 1

    cap.release()

# Function to process multiple videos and split for validation
def process_videos(video_dirs, train_output, val_output, split_for_val=False, val_ratio=0.2):
    for video_dir in video_dirs:
        video_files = [os.path.join(video_dir, f) for f in os.listdir(video_dir) if f.endswith('.mp4')]
        random.shuffle(video_files)

        if split_for_val:
            split_idx = int(len(video_files) * (1 - val_ratio))
            train_files = video_files[:split_idx]
            val_files = video_files[split_idx:]
        else:
            train_files = video_files
            val_files = []

        for video_path in train_files:
            process_video(video_path, train_output)

        for video_path in val_files:
            process_video(video_path, val_output)

# Process videos sequentially
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence"],
    output_paths["train_violence"],
    output_paths["val_violence"],
    split_for_val=True
)
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence"],
    output_paths["train_nonviolence"],
    output_paths["val_nonviolence"],
    split_for_val=True
)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



0: 640x640 7 persons, 323.1ms
Speed: 2.8ms preprocess, 323.1ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 294.4ms
Speed: 3.4ms preprocess, 294.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 persons, 1 baseball glove, 313.2ms
Speed: 2.7ms preprocess, 313.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 291.6ms
Speed: 2.5ms preprocess, 291.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 persons, 292.1ms
Speed: 3.4ms preprocess, 292.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 person, 1 bicycle, 235.8ms
Speed: 2.9ms preprocess, 235.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 223.6ms
Speed: 2.8ms preprocess, 223.6ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 222.7ms
Speed: 2.5ms preprocess, 222.7ms 

In [7]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to the directories containing extracted frames for training and validation
train_dir = "/kaggle/working/extracted_frames/train"
val_dir = "/kaggle/working/extracted_frames/val"

# Parameters
IMG_SIZE = (224, 224)  # Target image size for resizing
BATCH_SIZE = 32  # Number of images per batch during training/validation

# 1. Data Augmentation and Preprocessing
# Training Data Augmentation (train_datagen)
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values from [0, 255] to [0, 1]
    rotation_range=20,  # Random rotation within ±20 degrees
    width_shift_range=0.2,  # Random horizontal shift (20% of image width)
    height_shift_range=0.2,  # Random vertical shift (20% of image height)
    shear_range=0.2,  # Random shearing transformations
    zoom_range=0.2,  # Random zoom (in or out) by up to 20%
    horizontal_flip=True,  # Random horizontal flip
    brightness_range=[0.8, 1.2],  # Randomly adjust image brightness
    fill_mode='nearest'  # Handle pixels outside image boundary using nearest pixel value
)

# Validation Data Preprocessing (val_datagen)
val_datagen = ImageDataGenerator(
    rescale=1.0/255  # Only rescale pixel values for validation (no augmentation)
)

# 2. Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,  # Resize images to the target size
    batch_size=BATCH_SIZE,  # Group images into batches of size BATCH_SIZE
    class_mode='binary',  # Binary classification (two classes: 0 or 1)
    shuffle=True  # Shuffle the data for training to ensure randomness
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,  # Resize images to the target size
    batch_size=BATCH_SIZE,  # Group images into batches of size BATCH_SIZE
    class_mode='binary',  # Binary classification for validation as well
    shuffle=False  # Do not shuffle for validation, keeping consistent ordering
)

# These generators can now be used in a model's fit method for training and validation.
# Example: cnn_model.fit(train_generator, validation_data=val_generator, epochs=10)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [8]:
import os

# Specify the path to the main directory
main_dir = '/kaggle/working/extracted_frames'

# Dictionary to store file counts
file_counts = {}

# Walk through each subdirectory and count files
for root, dirs, files in os.walk(main_dir):
    # Only count files, skip directories
    file_counts[root] = len(files)

# Print the file counts in each folder
for folder, count in file_counts.items():
    print(f"{folder}: {count} files")


/kaggle/working/extracted_frames: 0 files
/kaggle/working/extracted_frames/train: 0 files
/kaggle/working/extracted_frames/train/nonviolence: 0 files
/kaggle/working/extracted_frames/train/violence: 0 files
/kaggle/working/extracted_frames/val: 0 files
/kaggle/working/extracted_frames/val/nonviolence: 0 files
/kaggle/working/extracted_frames/val/violence: 0 files


In [13]:
from torchinfo import torchinfo

# Print model summary with sample input size
summary = torchinfo.summary(model, 
                          input_size=(1, 3, 224, 224),
                          col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"],
                          depth=4)
print(summary)

# If you don't have torchinfo installed, run:
# !pip install torchinfo

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
MobileNetClassifier                                     [1, 3, 224, 224]          [1, 1]                    1,615,848                 --                        --
├─Sequential: 1-1                                       [1, 3, 224, 224]          [1, 576]                  --                        --                        --
│    └─Sequential: 2-1                                  [1, 3, 224, 224]          [1, 576, 7, 7]            --                        --                        --
│    │    └─Conv2dNormActivation: 3-1                   [1, 3, 224, 224]          [1, 16, 112, 112]         --                        --                        --
│    │    │    └─Conv2d: 4-1                            [1, 3, 224, 224]          [1, 16, 112, 112]         432                       [3, 3]                    5,419,008
│    │  